In [ ]:
%pip install buckaroo
try:
    from google.colab import output
    output.enable_custom_widget_manager()
except Exception as e:
    print(e)

In [ ]:
import numpy as np
import itertools
import pandas as pd
import buckaroo

In [ ]:
def bimodal(mean_1, mean_2, N, sigma=5, sigma2=5):
    X1 = np.random.normal(mean_1, sigma, int(N/2))
    X2 = np.random.normal(mean_2, sigma2, int(N/2))
    X = np.concatenate([X1, X2])
    return X
#ab = plt.hist(bimodal(20,40, 400),bins=10)

In [ ]:
def rand_cat(named_p, na_per, N):
    choices, p = [], []
    named_total_per = sum(named_p.values()) + na_per
    total_len = int(np.floor(named_total_per * N))
    if named_total_per > 0:
        for k, v in named_p.items():
            choices.append(k)
            p.append(v/named_total_per)

        choices.append(pd.NA)
        p.append(na_per/named_total_per)    
        return [np.random.choice(choices, p=p) for k in range(total_len)]
    else:
        return []

def random_categorical(named_p, unique_per, na_per, longtail_per, N):
    choice_arr = rand_cat(named_p, na_per, N)
    discrete_choice_len = len(choice_arr)

    longtail_count = int(np.floor(longtail_per * N))//2
    extra_arr = []
    for i in range(longtail_count):
        extra_arr.append("long_%d" % i)
        extra_arr.append("long_%d" % i)

    unique_len = N - (len(extra_arr) + discrete_choice_len)
    #print("discrete_choice_len", discrete_choice_len, "longtail_count", longtail_count, "unique_len", unique_len)
    for i in range(unique_len):
        extra_arr.append("unique_%d" % i)
    all_arr = np.concatenate([choice_arr, extra_arr])
    np.random.shuffle(all_arr)
    return all_arr        

In [ ]:
N = 4000
pd.DataFrame({
    'all_NA' :          [pd.NA] * N,
    'half_NA' :         random_categorical({1: .5}, unique_per=0,   na_per=.5, longtail_per=.0, N=N),
    'longtail' :        random_categorical({},      unique_per=0,   na_per=.2, longtail_per=.8, N=N),
    'longtail_unique' : random_categorical({},      unique_per=0.5, na_per=.0, longtail_per=.5, N=N),
})

In [ ]:
#kitchen sink
pd.DataFrame({
    'one': [1]*N, 
    'log_normal': np.random.lognormal(25, .3, N),
    'bimodal': bimodal(20,40, N),
    'cat_1':random_categorical({'foo': .35, 'bar': .2}, unique_per=.25, na_per=.2, longtail_per=0, N=N),
    'all_unique_cat':[str(i)+ "_a" for i in range(N)],
    'all_unique_N':[i for i in range(N)],
    'cat_long_tail': random_categorical({},      unique_per=0,   na_per=.2, longtail_per=.8, N=N),
    'all_NA': [pd.NA] * N,

    'cat_3':random_categorical({'foo': .3, 'bar': .05, 'baz':.1}, unique_per=.3, na_per=.25, longtail_per=0, N=N),
    'cat_4':random_categorical({'foo': .3, 'bar': .05, 'baz':.1}, unique_per=.3, na_per=.25, longtail_per=0, N=N)
})

In [ ]:
pd.DataFrame({
    'normal': np.random.normal(25, .3, N),
    'exponential' :  np.random.exponential(1.0, N) * 10 ,
    'increasing':[i for i in range(N)],
    'one': [1]*N, 
    'all_unique_cat': random_categorical({}, unique_per=1, na_per=0, longtail_per=0, N=N)
})

In [ ]:
pd.DataFrame({
    'bimodal' :  bimodal(20,40, N),
    'exponential' :  np.random.exponential(1.0, N) * 10 ,
    'geometric': np.random.geometric(.2, N) * 10,
    'log_normal': np.random.lognormal(25, .3, N),
    'normal': np.random.normal(25, .3, N),
})

In [ ]:
#categoricals
pd.DataFrame({
    'two_vals':                random_categorical({1: .75, 2: .25}, unique_per=0, na_per=0, longtail_per=0, N=N),
    'dominant_categories':     random_categorical({'foo': .6, 'bar': .25, 'baz':.15}, unique_per=0, na_per=0, longtail_per=0, N=N),
    'dispersed_categories':    random_categorical({'foo': .3, 'bar': .05, 'baz':.1}, unique_per=.3, na_per=.25, longtail_per=0, N=N),
})